In [1]:
!pip install dlib
!pip install face_recognition
!pip install opencv-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=6ff317b4454c93b764f986bd4af57fe8c714c36b620e86c829afcdab35e42ea5
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [2]:
import cv2
import numpy as np
import face_recognition

In [3]:
import os
import glob
import numpy as np
from datetime import datetime
import pickle
import time
import pytz

In [4]:
def extract_name(file_path):
    return file_path.split('/')[-1].split('.')[0]

In [5]:
def class_names(path):
  classes = []
  images = glob.glob(path + '/*.jpg')
  for image in images:
    name = extract_name(image)
    classes.append(name)

  return classes


In [6]:
classNames = class_names('/content/images')

In [7]:
type(classNames)

list

In [8]:
def find_embeddings(path):
  encodeed_list = []
  images = glob.glob(path + '/*.jpg')

  for img in images:
    img = cv2.imread(f'{img}')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    encoded_face = face_recognition.face_encodings(img)[0]
    encodeed_list.append(encoded_face)

  return encodeed_list

In [9]:
path = '/content/images'
encoded_face_train = find_embeddings(path)

In [10]:
type(encoded_face_train)

list

In [33]:
def markAttendance(name):
    csv_file = 'Attendance.csv'

    # Create the CSV file if it doesn't exist
    if not os.path.exists(csv_file):
        with open(csv_file, 'w') as f:
            f.write('Name,Time,Date\n')

    # Set the timezone to Asia/Dhaka (BDT)
    timezone_bdt = pytz.timezone('Asia/Dhaka')

    # Read existing data and update attendance
    with open(csv_file, 'r+') as f:
        myDataList = f.readlines()
        nameList = [entry.split(',')[0] for entry in myDataList]

        if name not in nameList:
            now_bdt = datetime.now(timezone_bdt)
            time_bdt = now_bdt.strftime('%I:%M:%S:%p')
            date_bdt = now_bdt.strftime('%d-%B-%Y')
            f.writelines(f'n{name}, {time_bdt}, {date_bdt}\n')

In [ ]:
# Later, read the list from the JSON fi

In [34]:
def call_attendence(img_path):

  img_path = img_path
  imgS = cv2.imread(f'{img_path}')
  imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    # Rest of your face recognition code...

  encoded_face = face_recognition.face_encodings(imgS)[0]

  matches = face_recognition.compare_faces([encoded_face_train], encoded_face)
  faceDist = face_recognition.face_distance(encoded_face_train, encoded_face)
  matchIndex = np.argmin(faceDist)
  #print(matchIndex)
  if matches:
    name = classNames[matchIndex].upper().lower()
    markAttendance(name)
    print("Done attendence for "+ str(name))

In [51]:
new_call_attendence('/content/test.jpg')

tesla has already been marked present today (10-March-2024).


In [49]:
def newmarkAttendence(name, date_today):
    csv_file = f'Attendance_{date_today}.csv'

    # Create the CSV file if it doesn't exist
    with open(csv_file, 'a+') as f:
        # Check if the file is empty (new file)
        if os.path.getsize(csv_file) == 0:
            f.write('Name,Time')

        # Read the existing data and update attendance
        f.seek(0)
        myDataList = f.readlines()
        nameList = [entry.split(',')[0] for entry in myDataList]

    # Set the timezone to Asia/Dhaka (BDT)
    timezone_bdt = pytz.timezone('Asia/Dhaka')

    if name.lower() in nameList:
        print(f"{name} has already been marked present today ({date_today}).")
    else:
        with open(csv_file, 'a') as f:
            time = datetime.now(timezone_bdt).strftime('%I:%M:%S:%p')
            f.writelines(f'\n{name}, {time}')
            print(f"Attendance marked for {name} on {date_today} at {time}.")

In [46]:
def new_call_attendence(img_path):

  img_path = img_path
  imgS = cv2.imread(f'{img_path}')
  imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    # Rest of your face recognition code...

  encoded_face = face_recognition.face_encodings(imgS)[0]

  matches = face_recognition.compare_faces([encoded_face_train], encoded_face)
  faceDist = face_recognition.face_distance(encoded_face_train, encoded_face)
  matchIndex = np.argmin(faceDist)
  #print(matchIndex)
  if matches:
    name = classNames[matchIndex].upper().lower()
    date_today = datetime.now().strftime('%d-%B-%Y')
    newmarkAttendence(name,date_today)
  else:
    print('Given photo is not of a listed employes ')


In [30]:
# take pictures from webcam
cap  = cv2.VideoCapture(0)
while True:
    success, img = cap.read()
    imgS = cv2.resize(img,(0,0),fx=0.25,fy=0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    faces_in_frame = face_recognition.face_locations(imgS)
    encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)
    for encode_face, faceloc in zip(encoded_faces,faces_in_frame):
        matches = face_recognition.compare_faces(encoded_face_train, encode_face)
        faceDist = face_recognition.face_distance(encoded_face_train, encode_face)
        matchIndex = np.argmin(faceDist)
        print(matchIndex)
        if matches[matchIndex]:
            name = classNames[matchIndex].upper().lower()
            y1,x2,y2,x1 = faceloc
            # since we scaled down by 4 times
            y1, x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img, (x1,y2-35),(x2,y2), (0,255,0), cv2.FILLED)
            cv2.putText(img,name, (x1+6,y2-5), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            markAttendance(name)
    cv2.imshow('webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [8]:


# Initialize webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()



while True:
    # Capture frame from webcam
    success, img = cap.read()

    # Check if img is not None
    if img is None:
        print("Error: Failed to capture frame.")
        continue  # Skip the rest of the loop and try again

    # Resize and process the image
    imgS = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    # Face recognition code...
    faces_in_frame = face_recognition.face_locations(imgS)
    encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)
    for encode_face, faceloc in zip(encoded_faces, faces_in_frame):
        matches = face_recognition.compare_faces(encoded_face_train, encode_face)
        faceDist = face_recognition.face_distance(encoded_face_train, encode_face)
        matchIndex = np.argmin(faceDist)
        print(matchIndex)
        if matches[matchIndex]:
            name = classNames[matchIndex].upper().lower()
            y1, x2, y2, x1 = faceloc
            # Since we scaled down by 4 times
            y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2-35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1+6, y2-5), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            markAttendance(name)

    # Display the image
    cv2.imshow('webcam', img)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


Streaming output truncated to the last 5000 lines.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capture frame.
Error: Failed to capt

KeyboardInterrupt: 

In [8]:
images = []
classNames = []
path = '/content/images'
mylist = os.listdir(path)
for cl in mylist:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

In [9]:
classNames

['Leo', 'Virat', 'bill', 'Tesla', 'Einstein', 'Shakib', 'elon']

In [19]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encoded_face = face_recognition.face_encodings(img)[0]
        encodeList.append(encoded_face)
    return encodeList
encoded_face_train = findEncodings(images)

[array([-0.16408046,  0.10435033,  0.02755699, -0.08288632, -0.19082892,
         0.07328326, -0.04082464, -0.02371554,  0.15852338, -0.02073427,
         0.24216282,  0.01061306, -0.31135279,  0.0105783 , -0.07339755,
         0.11908482, -0.19590993,  0.00356228, -0.09661292, -0.19969773,
         0.01200724,  0.08346271, -0.02765886,  0.05456619, -0.1184027 ,
        -0.22117642,  0.0264502 , -0.17873226,  0.05736259, -0.17321366,
         0.01765069,  0.02999161, -0.08504812, -0.0466086 ,  0.00885969,
         0.01577576, -0.04317319, -0.07809328,  0.16794248, -0.07025333,
        -0.05554868,  0.01466776,  0.08903591,  0.23829916,  0.17610039,
         0.03898719,  0.00618584, -0.08372051,  0.22395566, -0.25944895,
         0.08550392,  0.12738621,  0.11229209,  0.06224499,  0.05689788,
        -0.2434549 , -0.03659502,  0.16456889, -0.21905106,  0.08639339,
         0.08389033,  0.00496215, -0.09796619, -0.1005867 ,  0.20510328,
         0.18169333, -0.15749449, -0.17092922,  0.1